# Singly Linked List

In this challenge, create a data structure where `Nodes` will point to their children.

Required functionality:

- `.insert(value)` to insert a value
- `.remove(value)` to remove a value
- `.pop()` remove the first value and return it
- `.size()` to get the length of a list.
- `.search(value)` to get the index of a certain value
- `.display()` to see a string representation of the linked list

Bonus functionality:

- `.__str__` to get string representation of our list
- `.__len__` to get length of list
- `.__contains__` for `in` operations

There are two ways to solve this problem. One is by building a `Node` object _AND_ a `Container` object. OR. You could build just a `Node` object. I will be doing only the `Node` first.

In [1]:
class Node(object):
    
    def __init__(self, value, child=None):
        self.value = value
        self.child = child
        
    def insert(value):
        pass
    
    def remove(value):
        pass
        

Our first `__init__` method takes two parameters, one being optional.

`value` will be the data that this `Node` contains.
`child` will be our pointer to the next `Node`.

Let's fill in our insert function to spawn a `Node(value)` and set the child to that value.

In [2]:
class Node(object):
    
    def __init__(self, value, child=None):
        self.value = value
        self.child = child
        
    def insert(self, value):
        if self.child:
            self.child.insert(value)
        else:
            self.child = Node(value)
    
    def remove(value):
        pass
        

Let's break our code down:

```
1 | def insert(self, value):
2 |      if self.child:
3 |          self.child.insert(value)
4 |      else:
5 |          self.child = Node(value)
```

In line two, we ask our node the question: "Do you have a child?" If the current node does, we pass the question down to that node's child, and if that node has a child, it keeps passing the buck until we reach a point where there isn't a child. If there isn't a child, line 4 is triggered and we decend into the `else` block. Where we set the child to a new `Node` where the value is value we specify.

So does this work? Let's write some tests to find out.

In [3]:
def test_node_insert():
    SLL = Node(10)
    SLL.insert("a new node")
    SLL.insert("another node")
    assert SLL.value == 10
    assert SLL.child.value == "a new node"
    assert SLL.child.child.value == "another node"
    print("test_node_insert() passed!")
    
test_node_insert()

test_node_insert() passed!


Great! So let's continue with our assignment and write a `.remove()` method.

In [4]:
class Node(object):
    
    def __init__(self, value, child=None):
        self.value = value
        self.child = child
        
    def insert(self, value):
        if self.child:
            self.child.insert(value)
        else:
            self.child = Node(value)
    
    def remove(self, value):
        if self.value == value: # happens only if trying to remove root
            try:
                self.value = self.child.value
                self.child = self.child.child
                return True
            except AttributeError:
                return False
        elif self.child:
            if self.child.value == value:
                self.child = self.child.child
                return True
            else:
                return self.child.remove(value)
        else:
            return False
        
             
  

Let's break our code down:

```
01 | def remove(self, value):
02 |         if self.value == value: # happens only if trying to remove root
03 |             try:
04 |                 self.value = self.child.value
05 |                 self.child = self.child.child
06 |                 return True
07 |             except AttributeError:
08 |                 return False
09 |         elif self.child:
10 |             if self.child.value == value:
11 |                 self.child = self.child.child
12 |                 return True
13 |             else:
14 |                 return self.child.remove(value)
15 |         else:
16 |             return False
```

In line 2, we ask the question: "Is this node the node we want to remove?" If it is, we try do the following steps:
(Lines 4 - 6)
- Set this node's value to it's child value (which may throw an `AttributeError` if the child is `None`)
- Set this node's child to the child's child or grandchild
- `return True` to signal succesful completion of deletion
We wrap this code in a `try .. except` block incase the child is `None` (doesn't have a child), which means the user is trying to remove a Node that contains only one item. We'll return `False` in this scenario (line 8) to signal that user can't do this.

While we could raise an exception, I want to create a low-fuss system for now.

In the scenario where line 2 is `False` or the current node isn't the one we want to remove, we then ask the question (line 9): "Do we have a child?" If we do, then we check that child's value to see if it's the node we want to remove. If it is, we set the current node's child to the child's child.

In the case where the child isn't the node we want to remove, we pass the task to the that child and ask it the same questions.

Finally, in the scenario where the node doesn't have any children, we assume we reached the end of the list, and `return False` to signal that we weren't able to find the node the user wanted to remove

Let's see that remove works now.

In [5]:
def test_node_remove():
    SLL = Node(1)
    SLL.insert(3)
    SLL.insert(4)
    SLL.insert(5)
    SLL.insert(6)
    SLL.insert(7)
    
    SLL.remove(1)
    SLL.remove(6)
    SLL.remove(7)
    
    SLL2 = Node(1)
    SLL2.insert("hello")
    SLL2.remove(1)
    assert SLL2.remove("woops!") is False
    
    assert SLL2.value == "hello"
    assert SLL2.child == None
    
    assert SLL.value == 3
    assert SLL.child.value == 4
    assert SLL.child.child.value == 5
    assert SLL.child.child.child is None
    
    print("test_node_remove() passed!")
    
test_node_insert()
test_node_remove()

test_node_insert() passed!
test_node_remove() passed!


If it seems redundent to check the current node and the child node for deletion status over and over again, it's because it is. Let's time how slow our current method is.

In [6]:
def time_remove_node():
    history = []
    for x in range(100):
        import time
        Instance = Node("Hello World!")
        for x in range(100):
            Instance.insert(x)
        start_deletions = time.time()
        for x in range(100):
            Instance.remove(x)
        end_deletions = time.time()
        history.append(end_deletions - start_deletions)
    print("On average, it took {} seconds to remove 100 nodes.".format(sum(history)/len(history)))
    return sum(history)/len(history)
    
double_work = time_remove_node()

On average, it took 9.063720703125e-05 seconds to remove 100 nodes.


Let's see how much time we save by breaking up our `.remove()` method into two:

- `._remove()` for the recursion
- `.remove()` to initially check the root and then call `._remove()`

In [7]:
class Node(object):
    
    def __init__(self, value, child=None):
        self.value = value
        self.child = child
        
    def insert(self, value):
        if self.child:
            self.child.insert(value)
        else:
            self.child = Node(value)
    
    def _remove(self, value):
        if self.child:
            if self.child.value == value:
                self.child = self.child.child
                return True
            else:
                return self.child._remove(value)
        else:
            return False
        
    def remove(self, value):
        if self.value == value: # happens only if trying to remove root
            try:
                self.value = self.child.value
                self.child = self.child.child
                return True
            except AttributeError:
                return False
        else:
            return self._remove(value)
        

All we did was split the `.remove()` into two, let's run our tests and then check the time again.

In [8]:
test_node_remove()
two_methods = time_remove_node()
print("double_work to two_methods ratio is {}".format(double_work / two_methods))

test_node_remove() passed!
On average, it took 0.00013529539108276368 seconds to remove 100 nodes.
double_work to two_methods ratio is 0.6699208768745485


Interesting, it's actually faster to check the child and grandchild for _every_ node instead of splitting the method into two and doing half the work every time. Doing double_work is around 40% faster! This means we'll be sticking to one `.remove()` method, despite the redundancy.

Let's build out some more functionality, let's build `.pop()`.

An important thing to note, is that `.pop()` is supposed to be much faster than `.remove()`. So let's make it that way. We currently know from our tests it takes around `8.703947067260742e-05` to remove 100 nodes, so let's try to beat that record!


In [9]:
class Node(object):
    
    def __init__(self, value, child=None):
        self.value = value
        self.child = child
        
    def insert(self, value):
        if self.child:
            self.child.insert(value)
        else:
            self.child = Node(value)
    
    def remove(self, value):
        if self.value == value: # happens only if trying to remove root
            try:
                self.value = self.child.value
                self.child = self.child.child
                return True
            except AttributeError:
                return False
        elif self.child:
            if self.child.value == value:
                self.child = self.child.child
                return True
            else:
                return self.child.remove(value)
        else:
            return False
        
    def pop(self):
        try:
            value = self.value
            self.value, self.child = self.child.value, self.child.child
            return value
        except AttributeError:
            return

This code is a lot like our initial code of `.remove()`. We just moved it to `.pop()`. Let's write some tests for it, and then we can time it.

In [11]:
def test_node_pop():
    SLL = Node(0)
    for x in range(1, 100):
        SLL.insert(x)
    for x in range(0,100,-1):
        result = SLL.pop()
        assert result == x, "result: {} expected: {}".format(result, x)
    
    print("test_node_pop() passed!")
    
test_node_insert()
test_node_remove()
test_node_pop()

test_node_insert() passed!
test_node_remove() passed!
test_node_pop() passed!


In [12]:
def time_pop_node():
    history = []
    for x in range(100):
        import time
        Instance = Node("Hello World!")
        for x in range(100):
            Instance.insert(x)
        start_deletions = time.time()
        for x in range(100):
            Instance.pop()
        end_deletions = time.time()
        history.append(end_deletions - start_deletions)
    print("On average, it took {} seconds to pop 100 nodes.".format(sum(history)/len(history)))
    return sum(history)/len(history)

average_pop = time_pop_node()
print("Pop:Remove time ratio: {}".format(average_pop/double_work))

On average, it took 8.255958557128906e-05 seconds to pop 100 nodes.
Pop:Remove time ratio: 0.9108796296296295


It's faster, but not by much (9 measly percent). If we remove the `try .. except` block we could probably get it to go faster, but we need it in case the user screws up. 

Our next step is to implement a `.size()` method and port it over to `__len__`. We could do it one of two ways:

- Have a `._size` attribute that adjusts every time we call `.pop()`/`.insert()`/`.remove()`
- Calculate the size when `.size()` is called with iteration.

In terms of speediness and difficulty, I think I'll go with the first choice because it's easier to implement and it will be more time and memory efficient than loading it up every time we call `.size()`. So I'll just adjust some methods we have right now.

In [13]:
class Node(object):
    
    def __init__(self, value, child=None):
        self.value = value
        self.child = child
        self._size = 1
        
    def insert(self, value):
        if self.child:
            self._size += 1 # increase by one every insert
            self.child.insert(value) 
        else:
            self._size += 1
            self.child = Node(value)
            
    
    def remove(self, value):
        if self.value == value:
            try:
                self.value = self.child.value
                self.child = self.child.child
                self._size -= 1 # decrease by one every remove
                return True
            except AttributeError:
                return False
        elif self.child:
            if self.child.value == value:
                self.child = self.child.child
                self._size -= 1 # same thing
                return True
            else:
                return self.child.remove(value)
        else:
            return False
        
    def pop(self):
        try:
            value = self.value
            self.value, self.child = self.child.value, self.child.child
            self._size -= 1 # decrease one every pop
            return value
        except AttributeError:
            return
        
    def size(self):
        return self._size
    
    __len__ = size

Theoretically, we could make `size()` a `property` because it behaves so much like one, but the assignment calls for a method, and not a `@property` so we won't bother. It does however state a bonus method called `__len__` which we implemented. Which just a proxy to the `size()` method.

Let's write some tests.

In [14]:
def test_node_size():
    SLL = Node(5)
    for x in range(2, 10):
        SLL.insert(x)
        result = SLL.size()
        assert result == x, "result: {} expected: {}".format(result, x)
        assert len(SLL) == x
        
    for x in range(0, 10, -1):
        SLL.remove(x)
        result = SLL.size()
        assert result == x, "result: {} expected: {}".format(result, x)
        assert len(SLL) == x
        
    print("test_node_size() passed!")
    

test_node_insert()
test_node_remove()
test_node_pop()
test_node_size()

test_node_insert() passed!
test_node_remove() passed!
test_node_pop() passed!
test_node_size() passed!


Sweet! Let's tackle the next challenge: `.search()` and with that we can solve our `__contains__` bonus challenge.

In [18]:
class Node(object):
    
    def __init__(self, value, child=None):
        self.value = value
        self.child = child
        self._size = 1
        
    def insert(self, value):
        if self.child:
            self._size += 1 # increase by one every insert
            self.child.insert(value) 
        else:
            self._size += 1
            self.child = Node(value)
            
    
    def remove(self, value):
        if self.value == value:
            try:
                self.value = self.child.value
                self.child = self.child.child
                self._size -= 1 # decrease by one every remove
                return True
            except AttributeError:
                return False
        elif self.child:
            if self.child.value == value:
                self.child = self.child.child
                self._size -= 1 # same thing
                return True
            else:
                return self.child.remove(value)
        else:
            return False
        
    def pop(self):
        try:
            value = self.value
            self.value, self.child = self.child.value, self.child.child
            self._size -= 1 # decrease one every pop
            return value
        except AttributeError:
            return
        
    def size(self):
        return self._size
    
    __len__ = size
    
    def search(self, value):
        if self.value == value:
            return self
        elif self.child:
            return self.child.search(value)
        else:
            return False
    
    def __contains__(self, value):
        return bool(self.search(value))

The logic behind `.search()` is that it asks itself if is the value we're looking for. If it is, we return that Node; If it isn't, we proceed to ask the same question to each of it's children until we reach the end of the linked list and return False. our `__contains__` just takes advantage of `.search()` and returns the `bool()` value of the result.

Let's write some tests.

In [19]:
def test_node_search():
    SLL = Node(5)
    cities = "Seattle Edmonds Lynnwood Redmond Bellevue".split()
    for city in cities:
        SLL.insert(city)
        
        assert SLL.search(city).value == city
        assert city in SLL
        
    assert SLL.search("Wales") is False
    assert "Wales" not in SLL
    
        
    print("test_node_search() passed!")
    

test_node_insert()
test_node_remove()
test_node_pop()
test_node_size()
test_node_search()

test_node_insert() passed!
test_node_remove() passed!
test_node_pop() passed!
test_node_size() passed!
test_node_search() passed!


Last thing to do, is to create a `.__str__()` and `.display()` method for our Node object. let's write that out. I'm going to render our Node like so:

`[(0)>(1)>(2)>(3)]`

In [23]:
class Node(object):
    
    def __init__(self, value, child=None):
        self.value = value
        self.child = child
        self._size = 1
        
    def insert(self, value):
        if self.child:
            self._size += 1 # increase by one every insert
            self.child.insert(value) 
        else:
            self._size += 1
            self.child = Node(value)
            
    
    def remove(self, value):
        if self.value == value:
            try:
                self.value = self.child.value
                self.child = self.child.child
                self._size -= 1 # decrease by one every remove
                return True
            except AttributeError:
                return False
        elif self.child:
            if self.child.value == value:
                self.child = self.child.child
                self._size -= 1 # same thing
                return True
            else:
                return self.child.remove(value)
        else:
            return False
        
    def pop(self):
        try:
            value = self.value
            self.value, self.child = self.child.value, self.child.child
            self._size -= 1 # decrease one every pop
            return value
        except AttributeError:
            return
        
    def size(self):
        return self._size
    
    __len__ = size
    
    def search(self, value):
        if self.value == value:
            return self
        elif self.child:
            return self.child.search(value)
        else:
            return False
    
    def __contains__(self, value):
        return bool(self.search(value))
    
    def display(self):
        string = "["
        cursor = self
        while cursor:
            string += "({})>".format(cursor.value)
            try:
                cursor = cursor.child
            except AttributeError:
                break
        return string[:-1] + ']'
        
    __str__ = display

And of course, we write our tests!

In [26]:
def test_node_display():
    SLL = Node(1)
    assert SLL.display() == "[(1)]"
    assert str(SLL) == "[(1)]"
    SLL.insert(2)
    assert SLL.display() == str(SLL) == "[(1)>(2)]"
    SLL.insert("Norton Pengra")
    assert SLL.display() == str(SLL) == "[(1)>(2)>(Norton Pengra)]"
        
    print("test_node_display() passed!")
    

test_node_insert()
test_node_remove()
test_node_pop()
test_node_size()
test_node_search()
test_node_display()

test_node_insert() passed!
test_node_remove() passed!
test_node_pop() passed!
test_node_size() passed!
test_node_search() passed!
test_node_display() passed!


We have created a Singly Linked List! This concludes my tutorial. The github repo of this can be found [here](http://github.com/) 